In [ ]:
pip install selenium

In [ ]:
pip install pandas

In [1]:
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service = service, options = options)

url = "https://ww1.receita.fazenda.df.gov.br/DecVisualizador/Nfce/qrcode?p=53240400063960015718650040001523101629590693|2|1|1|8358b39453d2da3aadfe438eff48481f401f340b"

driver.get(url)


In [3]:

driver.switch_to.frame(driver.find_element(By.XPATH, "//iframe[@title='reCAPTCHA']"))

checkbox = WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, ".recaptcha-checkbox-border"))
)

# Clicando no checkbox do reCAPTCHA
checkbox.click()

# Esperando até que o checkbox do reCAPTCHA mude para o estado "check"
WebDriverWait(driver, 100000).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".recaptcha-checkbox-checked"))
)

driver.switch_to.default_content()

# Clicando no botão para ir para a página
button = driver.find_element(By.CLASS_NAME, "btn")
button.get_attribute("value")

button.click()



In [19]:
# Pegando a lista com os produtos
itens_str = driver.find_element(By.XPATH, "//ul").text

itens_lista = itens_str.split('\n')

itens_lista = list(filter(None, itens_lista))

# Convertendo a lista em um DataFrame do Pandas
df_all_itens = pd.DataFrame(itens_lista, columns=['Itens'])

# Pegando apenas os registros que tem "Cod: blabla" (indica que é um produto)
df_just_itens_name = df_all_itens[df_all_itens['Itens'].str.contains(r'\(Cód: \w+\)')].reset_index(drop=True)

# Limpeza para remover números e medidas dos nomes
df_just_itens_name['Itens'] = df_just_itens_name['Itens'].str.replace(r' \(Cód: S\d+\)', '', regex=True)
df_just_itens_name['Itens'] = df_just_itens_name['Itens'].str.replace(r'\d+[Xx]\d+\w*|\d+\w*|\d+', '', regex=True)
df_just_itens_name['Itens'] = df_just_itens_name['Itens'].str.strip()

# Pegando a primeira palavra de cada registro
df_just_itens_name['Primeira Palavra'] = df_just_itens_name['Itens'].str.split().str[0]

df_just_itens_name

,Itens,Primeira Palavra
0,AZEITE EV ESPANHOL,AZEITE
1,ESPAGUETE SEMOLA,ESPAGUETE
2,PASSATA POMODORO,PASSATA
3,QJ PARMESAO F AZUL C,QJ
4,MUSSARELA FATIADO,MUSSARELA
5,BURGUER FILE D COSTE,BURGUER
6,FILTRO DE PAPEL,FILTRO
7,DISH WASHER LAVA LOU,DISH
8,KETCHUP HEINZ .,KETCHUP
9,CERV QUILMES CLASSIC,CERV
